## 抓取財報資料並篩選

In [0]:
pip install twstock

     |████████████████████████████████| 1.9MB 2.7MB/s 


In [0]:
import twstock
import requests
import pandas as pd
import numpy as np
import datetime 
from bs4 import BeautifulSoup
import lxml

In [0]:
# 抓取財報資料函數
def fetch_financial_statements(year, season):
   # 查詢網址
    url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb06'
    
    # 查詢參數
    my_params = {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'isQuery': 'Y',
        'TYPEK':'sii',
        'year':str(year),
        'season':str(season)
    }
    req = requests.post(url,params = my_params)
    
    # 檢查HTTP Status
    if req.status_code != requests.codes.ok:
      print("FetchError"+req.status_code)
      return r.status_code
    else:
      soup = BeautifulSoup(req.text, "lxml") 
      # 把排版後的 html 印出來
      #print(soup.prettify()) 
      table=soup.find("table", class_= "hasBorder")
      df = pd.read_html(str(table))[0]
      #print(df)
      return df

In [0]:
# 抓取至今前三年各季財報資料

# 抓取今年及季
year_AD=datetime.datetime.today().strftime("%Y")
season_now=datetime.datetime.today().strftime("%m-%d")

# 第一季 5 月 16 日更新，半年度 8 月 15 日更新，第三季 11 月 15 日更新，年度 4 月 1 日更新。
if(season_now<"05-16"):
  season_now=0
elif(season_now<"08-05"):
  season_now=1
elif(season_now<"11-16"):
  season_now=2
elif(season_now>="11-16"):
  season_now=3
else:
  season_now=0
  
# 轉換成民國年份
year =int(year_AD)-1911;

list = []  

# 逐年逐季抓取資料
for yr in range(2):
  y=year-yr
  for season in range(3):
    s=season+1
    if(y!=year):
      df = fetch_financial_statements(y, s)
      list.append(df)
    else:
      if(s<season):
        df = fetch_financial_statements(y, s)
        list.append(df)

list

[        0        1          2                    3                      4  \
 0    公司代號     公司名稱  營業收入(百萬元)  毛利率(%)(營業毛利)/(營業收入)  營業利益率(%)(營業利益)/(營業收入)   
 1    1101       台泥   24179.21                23.95                  19.23   
 2    1102       亞泥   16573.88                20.09                  16.13   
 3    1103       嘉泥     471.26                11.63                 -10.75   
 4    1104     環球水泥    1166.10                11.88                   4.24   
 5    1108     幸福水泥     832.71                 2.47                  -4.15   
 6    1109     信大水泥    1061.38                22.71                  13.71   
 7    1110       東泥     412.49                 8.35                   1.83   
 8    1201     味全公司    3917.35                28.20                  -0.59   
 9    1203     味王公司    1447.47                33.75                  18.05   
 10   1210     大成長城   17123.12                14.90                   3.42   
 11   1213       大飲     149.09                10.74             